### Information Value & Weight of Evidence(정보가치와 근거의 무게)

Kaggle이나 데이터분석을 하다 보면 모델의 성능을 높이기 위해 여러가지 Feature를 만들어낸다.
그러나 Feature를 무조건 적으로 많이 만든다고해서 성능이 올라가는 것은 아니다. Target에 대한 영향력이 큰 Feature들이어야 성능에 영향을 줄 수 있는데,
그렇다면 영향력이 큰 Feature라는 것을 어떻게 평가할 것인가?

Feature Selection 하는 방법들을 보면 GBM 모델들의 Feature Importance 이용하거나 Decision Tree 나 RandomForest 의
Classifier 객체의 Feature importance_ 메서드를 활용해 비교하는 방법이 자주 보인다.

하지만 또 다른 방법으로, Information Value를 이용한 Feature Selection 기법도 존재한다.이번 글에선 information Value를 이용한 Feature selection 방법을 소개하려고 한다.

WOE와 IV는 로지스틱 회귀 기법에서 파생된 기법으로 신용평가분야에서 4-5년 이상 존재해오던 개념으로 채무 불이행 발생 확률과 같은 신용위험평가 모델링 과제에서 변수를 선별하기 위한 벤치마칭 자료로 사용되어 왔다. 두 개념은 데이터를 탐색하고 변수를 선별하는데 사용되며, 이항 분류 모델에 대한 탐색적 데이터 분석과 가변적 변서 선별을 위한 훌륭한 프레임워크를 제공한다. WOE는 독립변수와 종속변수 사이의 관계를 설명하는 지표이며, IV는 그 설명의 강도를 측정하는 값이다. 자세한 내용은 아래와 같이 정리하였다.

#### 1.Weight of Evidence(WOE)란?

WOE(근거의 무게)는 종속변수와 관련하여 영향력 있는 독립변수의 예측력을 나타내는 지표다. 기본적이론은 신용 평가 모델에서 발전한 것이기 때문에 일반적으로 좋고 나쁜 고객의 분리 측정의 기준으로 설명된다.

좋은 고객이란 신용채무능력이 가능한고객, 나쁜 고객이란 신용채무능력이 불가한 고객을 의미한다. 일반적으로 표현하자면 좋은 고객은 Non-events라 표현할 수 있고, 나쁜 고객은 Events라 표현할 수 있다. WOE를 구하는 수식을 보면 다음과 같다.

$$ WOE=ln\frac{Distribution of Goods}{Distibution of Bads}=ln\frac{\% of non-events}{\% of events}$$

먼저 WOE값의 결과로 얘기하자면 값이 양수이면 좋은 고객의 비율이 나쁜 고객의 비율보다 큰큰 것 의미하고, 음수이면 나쁜 고객의 비율이 큰 것을 의미한다.

WOE의 계산 순서는 다음과 같다.

1.	연속형 변수인 경우, 데이터를 10개의 구간으로 Binning 한다. (또는 분포에 다라 분할 구간이 달라질 수 있으나 통상적으로 10개의 구간으로 분할)
2.	각 그룹의 이벤트, 비이벤트의 수를 계산
3.	각 그룹에서 이벤트 비율과 비이벤트 비율을 계산
4.	비이벤트 비율과 이벤트 비율을 나누어 자연 로그를 취하여 각 구간별 WOE를 계산
5.	범주형 변수의 경우 분할된 구간으로 WOE를 계산

WOE는 종속 변수의 분포 유사성(즉, 사건 및 비사건의 수)에 기초하여 연속적인 독립변수를 하나의 범주로 변환하는데 사용된다. 연속형 독립변수의 경우 10 또는 20개로 분할된 그룹의 WOE를 계산하여 유사한 WOE 값을 가진 범주를 결합하고 범주의 구분을 WOE 값으로 대체하여 사용가능하며,
범주형 독립변수의 경우 각 범주의 WOE를 계산하고 유사한 WOE를 가진 범주를 재그룹핑하여 범주의 구분의 기존의 구분값이 아닌 WOE를 사용하여 기존의 범주가 아닌 연속형 변수로 변환하여 사용할 수 있다. WOE값이 유사하다는 것은 사건, 비사건의 비율이 동일하여 각 범주의 형태가 동일하다고 볼 수 있기 때문에 같은 범주로 합쳐 구분할 수 있는 것이다.

다시 정리를 하자면 WOE의 각 범주에는 관측치의 최소 5%이상의 값이 포함되어야 하며, 비사건과 사건의 데이터 수가 0이 아니어야 한다. 또한 각 범주마다 구별되어야 하고, 유사 집단은 통합하여 범주의 단순성을 고려하여 오버 피팅에 강한 변수를 선택하여야 한다. 또한 결측 값에 대하여 따로 범주화하여 구간을 분리하므로 결측치에 대한 특별한 보정이 없이도 사용이 가능한 장점을 가지고 있다.

앞서 말한 범주를 구분하는 기준은 일반적으로 10개 또는 20개의 범주로 분할하며, 이상적으로는 각 범주에 5%이상의 경우가 포함되어야 한다. 데이터에서 중요한 패턴을 포착하는 범주가 적을수록 데이터의 잡음이 없어지게 되고, 케이스가 5% 미만인 범주는 실제 데이터의 분포의 특징을 반영하지 못할 수 있으므로, 모델의 불안정성을 야기할 수 있다. 특정 범주의 케이스 값들이 0인 경우 0.5를 추가하여 Adjusted WOE를 계산하여 사용할 수 있다.
WOE는 연속형 범주를 범주화해주는 방법으로 사용되어 이상치를 처리하는데 효과적이며, 결측치를 다른 그룹으로 분할하여 관리할 수 있다. 또한 범주형 변수로 치환하므로 더미변수생성이 필요 없고, 로그비를 통한 엄격한 선형관계를 구축하는데 도움이 되어 변수의 영향도를 설명하기 쉬운 지표로 사용할 수 있다.
정리하자면 WOE는 종속변수를 설명하는 단변량 독립변수의 각 범주별 영향력(가중치)를 판단하는 값이고, 연속형 변수의 구간을 분할하여 각 구간별로 변화하는 종속변수에 대한 증거값을 판단하는 지표로, 로그비를 사용하여 그룹간 상대적인 영향력 또는 효과의 차이가 있다는 것을 증명해주는 지표로 사용될 수 있다.









